<a href="https://colab.research.google.com/github/KseniyaBarinova/Datasets/blob/main/Exam_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Скрипт выполняет классификацию текста с помощью предобученной модели на заданном датасете и выводит отчет о классификации с метриками, такими как точность, полнота и F1-мера

In [10]:
!pip install datasets transformers accelerate evaluate scikit-learn

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from sklearn.metrics import classification_report

from transformers import BertTokenizer, BertForSequenceClassification
import evaluate
import torch
from torch.utils.data import DataLoader
from transformers import TextClassificationPipeline
from datasets import load_dataset

In [12]:
# Загрузка предобученного BERT
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# Загрузка датасета
dataset = load_dataset("glue", "sst2")
train_dataset = dataset["train"]
test_dataset = dataset["validation"]

In [14]:
# Предобработка данных
def preprocess(example):
    return tokenizer(example["sentence"], padding="max_length", truncation=True, return_tensors="pt")


In [15]:
train_dataset = train_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)

In [16]:
# Создание DataLoader'ов
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

In [17]:
# Функция для оценки качества модели
def evaluate(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in data_loader:
            inputs = {key: val.squeeze(1) for key, val in batch.items()}
            labels = inputs.pop("labels")
            outputs = model(**inputs)
            _, predicted = torch.max(outputs.logits, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total


In [18]:
# Оценка качества на тестовом датасете
test_accuracy = evaluate(model, test_loader)
print("Test Accuracy:", test_accuracy)

AttributeError: 'list' object has no attribute 'squeeze'